# Import

In [1]:
import os
import argparse
import numpy as np
# This test mirrors the SHARP pipeline and now processes the full capture by default.
# Equation references (see apply_cfr_equations.py for detailed comments):
# - Load CFR/CSI (measured \bar H): CFR model and offsets
# - Build indices/frequencies: Eq. (6)
# - Phase sanitization via sparse model h = T r and Lasso: Eq. (5), (8), (9)-(11), (14)-(15)
# - Doppler (Hann+FFT over time, sum across subcarriers): Eq. (19)-(21)
# - Velocity mapping Δv (bin spacing): Eq. (22)
from apply_cfr_equations import (
    load_esp32_csv_lltf51_simple as load_esp32_csv,
    lltf51_indices, 
    reorder_or_pad,
    build_freq_vector_from_indices,
    sanitize_phase,
    REPO_ROOT,
)
from plot_doppler import plot_doppler

# Prepare files

In [ ]:
INPUT_DIR = os.path.join(REPO_ROOT, 'input_files') 
OUT_DIR = os.path.join(REPO_ROOT, 'output_files')

# Ensure the top-level output directory exists
os.makedirs(OUT_DIR, exist_ok=True)

# For each sub-folder in input_files, sanitize and save results to a matching
# sub-folder in output_files.
for folder in os.listdir(INPUT_DIR):
    folder_path = os.path.join(INPUT_DIR, folder)
    if os.path.isdir(folder_path):
        # Mirror the folder structure inside the output directory
        out_folder = os.path.join(OUT_DIR, folder)
        os.makedirs(out_folder, exist_ok=True)

        for file in os.listdir(folder_path):
            if file.endswith('.csv'):
                file_path = os.path.join(folder_path, file)

                # Load raw CSI, sanitize phase, and save
                H_raw, ts = load_esp32_csv(file_path)
                idx = lltf51_indices()
                H = reorder_or_pad(H_raw, idx)
                freq_vec = build_freq_vector_from_indices(idx)
                H_sanit = sanitize_phase(H, freq_vec)

                np.savez_compressed(os.path.join(out_folder, file), H=H_sanit, ts=ts)
                print(f"Sanitized {file_path} -> {os.path.join(out_folder, file)}.npz")

Sanitized c:\Users\ggad\Desktop\SHARP\input_files\test\drink2.csv -> c:\Users\ggad\Desktop\SHARP\output_files\test\drink2.csv.npz
Sanitized c:\Users\ggad\Desktop\SHARP\input_files\test\eat.csv -> c:\Users\ggad\Desktop\SHARP\output_files\test\eat.csv.npz
Sanitized c:\Users\ggad\Desktop\SHARP\input_files\test\empty.csv -> c:\Users\ggad\Desktop\SHARP\output_files\test\empty.csv.npz
Sanitized c:\Users\ggad\Desktop\SHARP\input_files\test\smoke2.csv -> c:\Users\ggad\Desktop\SHARP\output_files\test\smoke2.csv.npz
Sanitized c:\Users\ggad\Desktop\SHARP\input_files\test\watch.csv -> c:\Users\ggad\Desktop\SHARP\output_files\test\watch.csv.npz
